# Filter Itemid Prescriptions

First, we find the number of observation that has each unit of measurement. If the top frequent one has the observations less than 90% of total, then we discard it.

Also, there are many medicine that never be used by any patients, so we also discard it.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count

from utils import getConnection

%matplotlib inline

In [2]:
conn = getConnection()
    
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [3]:
def stat_prescript_unit_task(i, admission_ids_txt):
    conn = getConnection()
    # foreach medicine, list the dose unit of medicine and the count of observations that use that unit.
    cur = conn.cursor()
    cur.execute('SELECT dose_unit_rx, count(dose_unit_rx) FROM mimiciii.prescriptions WHERE formulary_drug_cd = \''+ str(i) +'\' and hadm_id in (select * from admission_ids) group by dose_unit_rx')
    outputunits = cur.fetchall()
    
    # sort it descendently
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    return (i, outputunits)

p = Pool(cpu_count() * 2)
results = [p.apply_async(stat_prescript_unit_task, args=(i, admission_ids_txt)) for i in prescript_itemid]
p.close()
p.join()

results = [x.get() for x in results]
np.save('res/filtered_prescript_raw.npy', {'raw': results})
print('saved!')

saved!


In [3]:
valid_prescript =[]
valid_prescript_unit =[]
dropped_id = []
notfound = []
results = np.load('res/filtered_prescript_raw.npy').tolist()['raw']

for x in results:
    i, outputunits = x[0], x[1]
    # check if medicine is never used by anybody, then discard it.
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
#         print("\n**PRES DROPPED "+str(i))
        notfound.append(i)
        for o in outputunits:
            print(o)
        print('')
        continue
        
    # calculate the percentage of observation of main unit.
    percentage = float(outputunits[0][1]) / total *100.
    if(percentage < 90): 
        # never drop the list A
#         if(i in manual_valid):
#             print("\n\n****PRES NOT DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
        
        # drop it !
#         else:
            dropped_id.append(i)
            print("\n\n****PRES DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
            continue
#     print("PRES "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits))
    
    # keep it and also save the main unit of it.
    valid_prescript.append(i)
    valid_prescript_unit.append(outputunits[0][0])




****PRES DROPPED SYRG1 : 75.53 : 6 : [('SYR', 3966), ('mL', 861), ('ml', 406), ('VIAL', 7), ('mg', 6), ('AMP', 5)]



****PRES DROPPED LIDO130I : 51.61 : 4 : [('ml', 16), ('mL', 14), ('dose', 1), (None, 0)]



****PRES DROPPED D5W500 : 60.14 : 3 : [('mL', 7475), ('ml', 4954), ('SYR', 1)]



****PRES DROPPED ETHYT : 75.00 : 2 : [('SPRY', 3), ('Appl', 1)]



****PRES DROPPED PCASYR : 64.52 : 3 : [('ml', 140), ('mL', 74), ('SYR', 3)]



****PRES DROPPED MPRED0 : 85.00 : 3 : [('mg', 17), ('dose', 2), ('MD to order daily dose', 1)]



****PRES DROPPED SYRINGEIND : 59.70 : 6 : [('ml', 40), ('dose', 10), ('mL', 8), ('SYR', 7), ('VIAL', 1), ('AMP', 1)]



****PRES DROPPED SEL25T : 63.64 : 2 : [('ml', 7), ('mL', 4)]



****PRES DROPPED DARB60SY : 50.00 : 3 : [('mcg/ml', 3), ('mcg/mL', 2), ('mcg', 1)]



****PRES DROPPED NSSYR : 88.24 : 2 : [('ml', 30), ('SYR', 4)]



****PRES DROPPED KCLBASE : 67.96 : 2 : [('mL', 6029), ('ml', 2842)]



****PRES DROPPED NS45500 : 54.17 : 2 : [('mL', 26), ('m

In [6]:
print(valid_prescript, valid_prescript_unit)
np.save('res/filtered_prescript.npy',{'id':valid_prescript,'unit':valid_prescript_unit})
print('saved!')

['DXIG40I', 'NORESTFE', 'ARIP10', 'BACI5I', 'CALCITVITD', 'D12.5W250I', 'NEOLOPE2L', 'METR500I', 'LACO50L', 'OLME20', 'SILD25', 'MYCO250IND', 'ROPI5', 'NEOSYRD12.5W', 'VALDIND', 'GLAR100I', 'GAST30L', 'MIDA50I', 'VANCOINV', 'CORTIND', 'RALT400IND', 'VANC500L', 'LANT750', 'METO25', 'BACTDS', 'DIAZE5I', 'ACET100D', 'MORP20/30L', 'MAGSUL20I', 'LITH450SA', 'DOXA1', 'CORTOTIC', 'ENOX60I', 'PROP100IG', 'FENT100P', 'LEUC5', 'VANC.000125L', 'MYCO2C', 'DISU250', 'FLUC60L', 'ACET800I', 'SNLR260O', 'CLAR500SYR', 'DICL25', 'NS20SYR', 'HYDR1O', 'CHLO2', 'SORA200', 'VITE100', 'COMBPTCH', 'NTG100PM', 'OXYB5', 'CYCLOBASE', 'GOLY4L', 'ISMO10', 'MICO745C', 'DRON400', 'LATA5OS', 'FOSA150IV', 'DARU400', 'LAMO100BRAND', 'NIFR150', 'FENT20I', 'GLUC5XL', 'ENOX120I', 'TEMO100', 'CISA10I', 'HYDZ25', 'FURO40', 'LIDO100S', 'GLYCPED12', 'IRON25I', 'BACT10I', 'HYDR12.5LF', 'HALD5I', 'HALO10', 'TELA250I', 'ZONI10L', 'DOXA4', 'PHEN65I', 'SEND6SYR', 'NEOAQUA', 'TOBR3ES', 'TRDI40I', 'IRON100I', 'TRETL', 'CLINBASE', 'F